In [1]:
# load libraries
from gurobipy import *
import math as m
import numpy as np
import pandas as pd
from typing import List,Tuple


In [2]:
#describing single break patterns:

def hb(k:int, r:int) -> int:
    """for home-away pattern with single home-break at position k,
    yield pattern at position r, where 1 denotes home, 0 denotes away."""
    return (int(r>=k) + r + k)%2

def ab(k:int, r:int) -> int:
    """for home-away pattern with single away-break at position k,
    yield pattern at position r, where 1 denotes home, 0 denotes away."""
    return 1 - hb(k,r)

def bdifs(N: int) -> List[int]:
    """Generate default break difference pattern with evenly split {1,3} patterns
    of the form: 1,3,2,2,1,2,(3,1)^k,2,(1,3)^m.
    Here k = m+1 or k=m+2, depending on value of N. N/2 = 7 + 2(k-m)+4m."""
    # set breakdif pattern
    nr31pairs = (N-10)//4
    nrlow = nr31pairs//2
    assert nrlow >= 1, "wrong input, N too small"
    nrhigh = nr31pairs - nrlow
    brkdif = [1,3,2,2,1,2]
    for i in range(nrhigh):
        brkdif.append(3)
        brkdif.append(1)
    brkdif.append(2)
    for i in range(nrlow-1):
        brkdif.append(1)
        brkdif.append(3)
    return brkdif

def brnds(difs: List[int]) -> List[Tuple[int]]:
    """Given a list of break diffs yields a list of round ids coupled 
    with a break type, where the first break is assumed to be in round 0, 
    and of type 0."""
    #building the cyclic ordering
    brp = [(0,0)]
    lastpair = (0,0)
    for bd in difs[1:]:
        nxt = (lastpair[0]+bd,(lastpair[1]+bd)%2)
        brp.append(nxt)
        lastpair = nxt
    nxt = (0,1)
    brp.append(nxt)
    lastpair=nxt
    for bd in difs[1:]:
        nxt = (lastpair[0]+bd,(lastpair[1]+bd)%2)
        brp.append(nxt)
        lastpair = nxt
    return brp
    
def hsgn( b:Tuple[int], r:int)->int:
    """signature of brk pattern b = (rnd,type) applied to 
    associated player p>=0 for round r>=0."""
    rnd,tp = b[0],b[1]
    if tp==0:
        return hb(rnd,r)
    else:
        return ab(rnd,r)
    
def sx(i: int) -> int:
    if i==5:
        return 6
    else:
        return i
    

In [3]:
# pre-arranging team-break matches, in order to speed up schedule construction
# pre-fixed schedules upto N=30
# Idea is to split teams in 4 almost equal sized subsets of consecutive numbers (quarters)
# while reserving 14 numbers that serve as glue inbetween quarters, 14 = a + b + c
# quartile blocks appear in schedule in order Q3,Q1,Q2,Q4 
# and may be reversed and/or pairwise flipped

def pairflip(listIN: List[int]) -> List[int]:
    """pair wise flip consecutive elements in list.""" 
    reslist = listIN.copy()
    for i in range(0,len(listIN),2):
        reslist[i],reslist[i+1] =  reslist[i+1],reslist[i]
    return reslist

def tmbrks(N: int, part:bool = False, a:int=0, b:int=0,c:int=0,allfree: bool=False) -> List[Tuple[int]]:
    """Assign team ids to breaks as they apparently allow a fair
    SRR with minimal breaks."""
    assert N >= 18 and N%4==2, "wrong input, N too small, odd, or a multiple of 4"
    
    res18 = [(14, 0, 0) , (1, 3, 1)  ,(15, 5, 1) , (9, 7, 1) , (6, 8, 0) , (12, 10, 0),
             (7, 13, 1) , (4, 14, 0) , (0, 16, 0) , (3, 0, 1) , (16, 3, 0), (2, 5, 0) ,  
             (8, 7, 0) , (11, 8, 1), (5, 10, 1), (10, 13, 0) , (13, 14, 1) , (17, 16, 1)]
    
    res22 = [(18, 0, 0), (1, 3, 1),  (19, 5, 1),  (11, 7, 1),  (6, 8, 0),  (16, 10, 0),  
             (15, 13, 1),  (14, 14, 0),  (7, 17, 1),  (4, 18, 0),  (0, 20, 0),  (3, 0, 1),  
             (20, 3, 0),  (2, 5, 0),  (10, 7, 0),  (13, 8, 1),  (5, 10, 1),  (8, 13, 0),  
             (9, 14, 1),  (12, 17, 0),  (17, 18, 1),  (21, 20, 1)  ]
    
    res26 =[(22, 0, 0),  (1, 3, 1),  (23, 5, 1),  (13, 7, 1),  (8, 8, 0),  (18, 10, 0),  
            (17, 13, 1),  (16, 14, 0),  (9, 17, 1),  (6, 18, 0),  (20, 20, 0),  (21, 21, 1),  
            (0, 24, 0),  (3, 0, 1) , (24, 3, 0) ,  (2, 5, 0) ,  (12, 7, 0) ,  (15, 8, 1),  
            (7, 10, 1),  (10, 13, 0),  (11, 14, 1),  (14, 17, 0),  (19, 18, 1),  (5, 20, 1) ,  
            (4, 21, 0),  (25, 24, 1)]
    
    res30 = [(26, 0, 0),   (1, 3, 1),   (27, 5, 1),   (17, 7, 1),   (14, 8, 0),   (24, 10, 0),   
             (23, 13, 1),   (22, 14, 0),   (21, 17, 1),   (20, 18, 0),   (15, 21, 1),   (8, 22, 0),  
             (4, 24, 0),   (5, 25, 1),   (0, 28, 0),  (3, 0, 1),   (28, 3, 0),   (2, 5, 0) ,   
             (16, 7, 0),   (25, 8, 1),   (7, 10, 1),   (10, 13, 0),   (11, 14, 1),   (12, 17, 0),   
             (13, 18, 1) ,   (6, 21, 0) ,   (9, 22, 1),   (19, 24, 1),   (18, 25, 0),   (29, 28, 1)] 
    
    res30a = [(26, 0, 0), (1, 3, 1), (27, 5, 1), 
             (15, 7, 1), (12, 8, 0), (24, 10, 0), 
             (19, 13, 1), (18, 14, 0), (17, 17, 1), (16, 18, 0), 
             (13, 21, 1), (6, 22, 0), (22, 24, 0), (25, 25, 1), 
             (0, 28, 0), (3, 0, 1), (28, 3, 0), (2, 5, 0), 
             (14, 7, 0), (21, 8, 1), (5, 10, 1), 
             (8, 13, 0), (9, 14, 1), (10, 17, 0), (11, 18, 1), 
             (20, 21, 0), (23, 22, 1), (7, 24, 1), (4, 25, 0), 
             (29, 28, 1)]

    
    if N==18:
        return res18
    elif N==22:
        return res22
    elif N==26 and False:
        return res26
    elif N==30:
        return res30a
    elif N%8 == 2:
        #use spaces 4,2,8 between quartiles Q1,Q2,Q3,Q4
        # Q1  = 0,....,q1,  with q1 = 2*(N//8)-3
        # Q4  = N-1-q1,....,N-1
        # Q2  = q1+Q12+1,...,q1+Q12+L2, with L2 = (N-16-2q1)/2 = N/2 - 8 -q1
        # Q3  = q1+Q12+L2+Q23+1,...,q1+Q12+L2+Q23+L2
        # check : N-1-q1 - 1 - (q1+Q12+L2+Q23+L2) = N -2q1 -2 -Q12-Q23-2L2
        #   = N-2q1-2-Q12-Q23-N+16+2q1 = 14 - Q12 - Q23 = Q34
        
        q1 = 2*(N//8)-3
        if a==0:
            Q12 = 4
            Q23 = 2
            Q34 = 8
        else:
            Q12 = a
            Q23 = b
            Q34 = c
        
        brp = brnds(bdifs(N))
        
        # part refers to partial fix of team assignments; letting the intermediate sections free
        # allfree refers to no binding whatsoever
        if allfree:
            res = [ (-1,brp[i][0],brp[i][1]) for i in range(N) ]
            return res
        
        sec31 = (N-10)//8
        fst31 = (N-10)//4 - sec31  # now equals sec31
        if not part:
            res = [(N-4,0,0),(1,3,1),(N-3,5,1),(N//2-2,7,1),((N-2)//4,8,0),(2+3*(N-2)//4,10,0)]
        elif allfree:
            res = [(-1,0,0),(-1,3,1),(-1,5,1),(-1,7,1),(-1,8,0),(-1,10,0)]
        else:
            res = [(N-4,0,0),(1,3,1),(N-3,5,1),(-1,7,1),(-1,8,0),(-1,10,0)]
        curlen = 6
        
        Q3tms = list(reversed([N//2-7+Q12+Q23+i for i in range(2*fst31-2)]))
        Q3res = [ (Q3tms[i],brp[6+i][0],brp[6+i][1]) for i in range(2*fst31-2) ]
        res.extend(Q3res)
        curlen += len(Q3res)
        
        Q13tms = [ (N-2)//4+1, (N-2)//4-2, 3*((N-2)//4)-4, 3*((N-2)//4)-1 ]
        if not part:
            Q13res = [ (Q13tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(4) ]
        else:
            Q13res = [ (-1,brp[curlen+i][0],brp[curlen+i][1]) for i in range(4) ]
        res.extend(Q13res)
        curlen += 4
        
        Q1tms = pairflip(list(reversed([4+i for i in range(2*(sec31-2)) ])))
        Q1res = [ (Q1tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q1tms)) ]
        res.extend(Q1res)
        curlen += len(Q1res)
        
        if not part:
            res.extend([(0,N-2,0),(3,0,1),(N-2,3,0),(2,5,0)])
        elif allfree:
            res.extend([(-1,N-2,0),(-1,0,1),(-1,3,0),(-1,5,0)])
        else:
            res.extend([(0,N-2,0),(3,0,1),(N-2,3,0),(2,5,0)])
        curlen += 4
        
        Q31tms = [ 3*((N-2)//4)-2, 3*((N-2)//4)+1, (N-2)//4-1 ]
        if not part:
            Q31res = [ (Q31tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(3) ]
        else:
            Q31res = [ (-1,brp[curlen+i][0],brp[curlen+i][1]) for i in range(3) ]
        res.extend(Q31res)
        curlen += 3
        
        Q2tms = [ q1+Q12+1+i for i in range(2*fst31-2) ]
        Q2res = [ (Q2tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q2tms)) ]
        res.extend(Q2res)
        curlen += len(Q2res)
        
        if not part:
            Q341tms = [ 3*((N-2)//4), 3*((N-2)//4)+3 , 3*((N-2)//4)-3 , (N-2)//2-2]
            Q341res = [ (Q341tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q341tms)) ]
            res.extend(Q341res)
            curlen += len(Q341res)

            Q4tms = pairflip([ N-1-q1+i for i in range(2*sec31-4) ])
            Q4res = [ (Q4tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q4tms)) ]
            res.extend(Q4res)
            curlen += len(Q4res)
            
        else:
            Q341tms = [ -1,-1,-1,-1]
            Q341res = [ (Q341tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q341tms)) ]
            res.extend(Q341res)
            curlen += len(Q341res)
            
            Q4tms = pairflip([ N-1-q1+i for i in range(2*sec31-4) ])
            Q4res = [ (Q4tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q4tms)) ]
            res.extend(Q4res)
            curlen += len(Q4res)
            
        #finally add last team
        if not part:
            res.append((N-1,N-2,1))
        elif allfree:
            res.append((-1,N-2,1))
        else:
            res.append((N-1,N-2,1))
        
        return res   
        
    else:
        # N%8 == 6
         
        q1 = 2*(N//8)-3
        if a==0:
            Q12 = 4
            Q23 = 4
            Q34 = 6
        else:
            Q12 = a
            Q23 = b
            Q34 = c
        
        brp = brnds(bdifs(N))
        sec31 = (N-10)//8
        fst31 = (N-10)//4 - sec31  # now equals sec31 + 1
        if not part:
            res = [(N-4,0,0),(1,3,1),(N-3,5,1),(N//2,7,1),(N//2-3,8,0),(3*(N+2)//4,10,0)]
        elif allfree:
            res = [(-1,0,0),(-1,3,1),(-1,5,1),(-1,7,1),(-1,8,0),(-1,10,0)]
        else:
            res = [(N-4,0,0),(1,3,1),(N-3,5,1),(-1,7,1),(-1,8,0),(-1,10,0)]
        curlen = 6

        Q3tms = list(reversed([N//2-7+Q12+Q23+i for i in range(2*fst31-2)]))
        Q3res = [ (Q3tms[i],brp[6+i][0],brp[6+i][1]) for i in range(len(Q3tms)) ]
        res.extend(Q3res)
        curlen += len(Q3res)
        
        if not part:  
            Q13tms = [ N//2-2,(N+2)//4-2, 3*((N+2)//4)-2, 3*((N+2)//4)+1 ]  
        else:
            Q13tms = [-1,-1,-1,-1]
        Q13res = [ (Q13tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(4) ]
        res.extend(Q13res)
        curlen += 4
        
        Q1tms = pairflip(list(reversed([4+i for i in range(2*(sec31-2)) ])))
        Q1res = [ (Q1tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q1tms)) ]
        res.extend(Q1res)
        curlen += len(Q1res)
        
        if not part:
            res.extend([(0,N-2,0),(3,0,1),(N-2,3,0),(2,5,0)])
        elif allfree:
            res.extend([(-1,N-2,0),(-1,0,1),(-1,3,0),(-1,5,0)])
        else:
            res.extend([(0,N-2,0),(3,0,1),(N-2,3,0),(2,5,0)])
        curlen += 4
        
        if not part:
            Q21tms = [ N//2-1, 3*((N+2)//4)-3, (N+2)//4-3 ]
        else:
            Q21tms = [ -1,-1,-1 ]
        Q21res = [ (Q21tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(3) ]
        res.extend(Q21res)
        curlen += 3
            
        Q2tms = [ q1+Q12+1+i for i in range(2*fst31-2) ]
        Q2res = [ (Q2tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q2tms)) ]
        res.extend(Q2res)
        curlen += len(Q2res)
        
        if not part:
            Q2312tms = [ 3*((N+2)//4) - 4,3*((N+2)//4) - 1 ,(N+2)//4 - 1,(N+2)//4 - 4]
            Q2312res = [ (Q2312tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q2312tms)) ]
            res.extend(Q2312res)
            curlen += len(Q2312res)

            Q4tms = pairflip([ N-1-q1+i for i in range(2*sec31-4) ])
            Q4res = [ (Q4tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q4tms)) ]
            res.extend(Q4res)
            curlen += len(Q4res)
        else:
            Q2312tms = [ -1,-1,-1,-1]
            Q2312res = [ (Q2312tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q2312tms)) ]
            res.extend(Q2312res)
            curlen += len(Q2312res)
            
            Q4tms = pairflip([ N-1-q1+i for i in range(2*sec31-4) ])
            Q4res = [ (Q4tms[i],brp[curlen+i][0],brp[curlen+i][1]) for i in range(len(Q4tms)) ]
            res.extend(Q4res)
            curlen += len(Q4res)
            
        #finally add last team
        if not part:
            res.append((N-1,N-2,1))
        elif allfree:
            res.append((-1,N-2,1))
        else:
            res.append((N-1,N-2,1))
        
        return res
                  

In [4]:
def fpbreak(N: int,                     # number of teams, 2 modulo 4
            TmLim=0,                    # timelimit on MIP solver, in seconds
            mipgap=0.0,                 # allowed absolute integrality gap, improving BnB time
            xFix=[],                    # list of prefixed (i,j,r) assignment: brk i plays brk j in round r
            match=[],                   # list of (i,j, R) tuples: i plays j in some round r \in R
            part:bool = False,          # allows to not fix glue teams
            fixtm: bool = True,         # indicates compete tm-brk assignment, to strengthen MIP
            a:int = 0, b:int=0,c:int=0, # allows for alternative split between quarters
            allfree:bool = False,       # allows to skip team-brk pre-assignment
           ) -> (List[int],List[Tuple],np.ndarray):
    """Set up MIP model for size N for standardized break set cyclically ordered,
    assigning rounds to break-break-pairs and assigning teams to breaks, matching a 
    linear ordering on the breaks. By default take prefixed ordering.
    Return Teams-List, Breaks-List, and rounds matrix.
    """
    assert N>0 and N%4==2, "N should be positive and 2 times an odd number"
    Plys = range(N)
    Rnds = range(N-1)
    Brks = brnds(bdifs(N))  
    # a list of 2-tuples (r,p), of length N, with r in Rnds and parity p in {0,1}
    prehsgn = dict()
    for i in Plys:
        for r in Rnds:
            prehsgn[(Brks[i],r)] = hsgn(Brks[i],r)
        
    
    HalfPairs = [ (i,j) for i in range(N) for j in range(i+1,N) ]
    sched = Model("fairplay schedule for fixed breaks")
    
    HalfPairRndsInit = [(i,j,r) for (i,j) in HalfPairs 
                    for r in Rnds if (prehsgn[( Brks[i], r)] + prehsgn[( Brks[j], r)] == 1) ]
    
    tmslist = tmbrks(N,part,a=a,b=b,c=c,allfree=allfree)
    
    HalfPairRnds = set([ (i,j,r) for (i,j,r) in HalfPairRndsInit if (
    tmslist[i][0] < 0 or tmslist[j][0] < 0 or (
    (Brks[i][1]+Brks[j][1])%2 == 1 and prehsgn[( Brks[i], r)] == int(tmslist[i][0] < tmslist[j][0]) ) or ( 
    (Brks[i][1]+Brks[j][1])%2 == 0 and 1-prehsgn[( Brks[i], r)] == int(tmslist[i][0] < tmslist[j][0])
    ) )])
    
    
    
    x = sched.addVars(HalfPairRnds,vtype=GRB.BINARY, name="x")
    # x[i,j,r]=1 <-> Team(Brks[i]) plays against Team(Brks[j]) in round r
    
    sched.addConstrs((x[min(a,b),max(a,b),r] == 1 for (a,b,r) in xFix))
    
    pr = sched.addVars(HalfPairs,vtype=GRB.BINARY, name="pr")
    #pr[i,j] = 1 <-> Team(Brks[i]) < Team(Brks[j])
    
    # ij is played in some round
    sched.addConstrs((x.sum(i,j,"*") == 1 for i,j in HalfPairs))
    
    # j meets someone in each round
    sched.addConstrs((x.sum(j,"*",r) + x.sum("*",j,r) == 1 for r in Rnds for j in Plys))
    
    #linking i,j venue to pr
    for (i,j) in HalfPairs:
        Hrounds = [ r for r in Rnds if hsgn( Brks[i], r) == 1 and hsgn( Brks[j], r) == 0 ]
        if (Brks[i][1]+Brks[j][1])%2 == 1:
            #associated teams differ in parity
            sched.addConstr( pr[i,j] == quicksum([ x[i,j,r] for r in Hrounds if (i,j,r) in HalfPairRnds ]) )
        else:
            sched.addConstr( 1 - pr[i,j] == quicksum([ x[i,j,r] for r in Hrounds if (i,j,r) in HalfPairRnds ]) )
            
    #forcing the order pr[i,j] to a FULL ordering, that is no cycles a < b < c < a 
    #order N^3 constraints; not necessary if teams have been assigned already
    if part or allfree:
        sched.addConstrs(( pr[i,j] + pr[j,k] - pr[i,k] <= 1 
                      for i in range(N) for j in range(i+1,N) for k in range(j+1,N) ))
        sched.addConstrs(( 0 <= pr[i,j] + pr[j,k] - pr[i,k] 
                      for i in range(N) for j in range(i+1,N) for k in range(j+1,N) ))
    
    #for N <=30 just solve the above problem and do not apply the following tricks
    
    nr31pairs = (N-10)//4
    nrlow = nr31pairs//2
    nrhigh = nr31pairs - nrlow
    
    #notable boundaries : last 2-break
    last2brkrnd = N-2-4*(nrlow-1)
    last2brkid  = N//2 + 1 - 2*nrlow   #note always even
    
    if allfree:
        #fix adjacency of teames within quarter blocks
        
        #pair adjacency in fst quarter 6-latt2brkid-1
        thelen = last2brkid - 8
        fstone = 6
        lastone = fstone + thelen-2  - 1
        sched.addConstrs((pr[i,j] == pr[i,j+1] for j in range(fstone,lastone,2) for i in range(j)))
        sched.addConstrs((pr[j,i] == pr[j+1,i] for j in range(fstone,lastone,2) for i in range(j+2,N)))
        #total adjacency, actually [pair adjacency from fst+1]
        sched.addConstrs((pr[i,j] == pr[i,j+1] for j in range(fstone+1,lastone+1,2) for i in range(j)))
        sched.addConstrs((pr[j,i] == pr[j+1,i] for j in range(fstone+1,lastone+1,2) for i in range(j+2,N)))
        
        #pair adjacency in scnd quarter latt2brkid .. 
        fstone = last2brkid 
        lastone = fstone + thelen - 1 
        sched.addConstrs((pr[i,j] == pr[i,j+1] for j in range(fstone,lastone,2) for i in range(j)))
        sched.addConstrs((pr[j,i] == pr[j+1,i] for j in range(fstone,lastone,2) for i in range(j+2,N)))
        #flip adjacency
        sched.addConstrs((pr[i,j] == pr[i,j+3] for j in range(fstone+2,lastone-2,2) for i in range(j)))
        sched.addConstrs((pr[j,i] == pr[j+3,i] for j in range(fstone+2,lastone-2,2) for i in range(j+4,N)))
        
        #pair adjacency in thrd quarter 6+N//2 - latt2brkid-3+N//2
        fstone = 6 + N//2
        #lastone = fstone + thelen - 1
        #sched.addConstrs((pr[i,j] == pr[i,j+1] for j in range(fstone,lastone,2) for i in range(j)))
        #sched.addConstrs((pr[j,i] == pr[j+1,i] for j in range(fstone,lastone,2) for i in range(j+2,N)))
        lastone = fstone + thelen - 1
        sched.addConstrs((pr[i,j] == pr[i,j+1] for j in range(fstone,lastone,2) for i in range(j)))
        sched.addConstrs((pr[j,i] == pr[j+1,i] for j in range(fstone,lastone,2) for i in range(j+2,N)))
        #total adjacency
        sched.addConstrs((pr[i,j] == pr[i,j+1] for j in range(fstone+1,lastone+1,2) for i in range(j)))
        sched.addConstrs((pr[j,i] == pr[j+1,i] for j in range(fstone+1,lastone+1,2) for i in range(j+2,N)))
        
        #pair adjacency in frth quarter latt2brkid .. 
        fstone = last2brkid + N//2
        lastone = fstone + thelen - 1 
        sched.addConstrs((pr[i,j] == pr[i,j+1] for j in range(fstone,lastone,2) for i in range(j)))
        sched.addConstrs((pr[j,i] == pr[j+1,i] for j in range(fstone,lastone,2) for i in range(j+2,N)))
        #flip adjacency same as above
        sched.addConstrs((pr[i,j] == pr[i,j+3] for j in range(fstone+2,lastone-2,2) for i in range(j)))
        sched.addConstrs((pr[j,i] == pr[j+3,i] for j in range(fstone+2,lastone-2,2) for i in range(j+4,N)))
    
    # note the break also occurs in row lastbrkid + (N//2)
    # we have consecutive break difs 3,1,2,1  in rounds L2Br-3,L2Br-2,L2Br,l2Br+1
    # the associated rows are indexed L2Bid-2,L2Bid-1,L2Bid,L2Bid+1
    
    # we have five breaks associated with rows 1,2,3,4,5  (and with same rows shifted N//2)
    # these are associated with break difs 3,2,2,1,2   in rounds: 3,5,7,8,10
    
    ############## assume we enforce the found sequence as precedence #########
    
    #print(str(tmslist))
    sched.addConstrs(( pr[i,j] == int(tmslist[i][0] < tmslist[j][0]) 
                      for i,j in HalfPairs if ( tmslist[i][0] >= 0 and tmslist[j][0] >= 0)))
    if fixtm:
        sched.addConstrs((int(tmslist[i][0]) == quicksum([ pr[j,i] for j in range(i)]) +
                         quicksum([ 1 - pr[i,j] for j in range(i+1,N)])
                         for i in Plys if tmslist[i][0] >= 0 )) 
    
    #let us force meetings in odd rounds for all pairs of consecutive breaks
    #note that odd round 5 -> 6
    
    if N > 30:
        sched.addConstrs( x[r//2,(r+1)//2,sx(r)] == 1 for r in range(1,last2brkrnd-1,2)
                        if (r//2,(r+1)//2,sx(r)) in HalfPairRnds )
        sched.addConstrs( x[r//2+(N//2),(r+1)//2+(N//2),sx(r)] == 1 for r in range(1,last2brkrnd-1,2) 
                        if (r//2+(N//2),(r+1)//2+(N//2),sx(r)) in HalfPairRnds  )
        sched.addConstrs( x[r//2,(r+1)//2,r] == 1 for r in range(last2brkrnd-1,N,4)
                        if (r//2,(r+1)//2,r) in HalfPairRnds )
        sched.addConstrs( x[r//2+(N//2),(r+1)//2+(N//2),r] == 1 for r in range(last2brkrnd-1,N,4)
                        if (r//2+(N//2),(r+1)//2+(N//2),r) in HalfPairRnds )
        sched.addConstrs( x[(r+1)//2,(r+2)//2,r] == 1 for r in range(last2brkrnd,N-1,4)
                        if ((r+1)//2,(r+2)//2,r) in HalfPairRnds )
        sched.addConstrs( x[(r+1)//2+(N//2),(r+2)//2+(N//2),r] == 1 for r in range(last2brkrnd,N-2,4)
                        if ((r+1)//2+(N//2),(r+2)//2+(N//2),r) in HalfPairRnds )
        sched.addConstr( x[0,N-1,N-2] == 1 )
    
    #enforce matching of a team in a round as given in match list
    for matlist in match:
        rndid = matlist[0]
        rwid = matlist[1]
        candlist = [ (min(rwid,nwrw),max(rwid,nwrw),rndid) for nwrw in matlist[2:] ]
        LeftTups = [ (i,j,k) for i,j,k in candlist if (i,j,k) in HalfPairRnds]
        sched.addConstr( quicksum([x[i,j,k] for i,j,k in LeftTups]) == 1, name="match"+str(matlist))
  
    sched.update()
    
    #sched.write("fplayGame.lp")
    
    TimeLimit = TmLim
    if TimeLimit > 0:
        sched.setParam('TimeLimit', TimeLimit)   
    if mipgap > 0:
        sched.setParam('MIPGap',mipgap)
    
    sched.optimize()
    
    if sched.Status == GRB.INFEASIBLE:
        print("Infeasible")
        return [],[],[]
    
    if sched.Status != GRB.OPTIMAL:
        print("Not Optimal")
        #return [],[],[]
    
    tmass = []
    for i in range(N):
        pos = sum([ pr[j,i].X for j in range(i)]) + sum([ (1 - pr[i,j].X) for j in range(i+1,N)])
        tmass.append(int(pos))
    print("Team break assignment:")
    print(str(tmass))
    
    rndm = np.zeros([N,N],dtype=int)   #rounds for brkpairs
    tmrnds = np.zeros([N,N],dtype=int)   #rounds for team pairs
    tmopps = np.zeros([N,N-1],dtype=int)  # teams opponents in order 
    #now copy the x results
    for j,q,r in HalfPairRnds:
        if x[j,q,r].X > 0.5:
            rndm[j][q] = r
            rndm[q][j] = r
            tmrnds[ tmass[j]][tmass[q]] = r
            tmrnds[ tmass[q]][tmass[j]] = r
            tmopps[ tmass[j]][r] = tmass[q]
            tmopps[ tmass[q]][r] = tmass[j]
    
    if False:
        print("Team-Team-meet in Round:")
        for i in range(N):
            print(f'(Tm {i}): '+str(list(tmrnds[i])))

        print("Teams opponents:")
        for i in range(N):
            print(f'(Tm {i}): '+str(list(tmopps[i])))
      
    
    return tmass,Brks,rndm
    
    

In [5]:
def build_df(N:int, tms:List[int], brks:List[Tuple], rnds:np.ndarray):
    """Create table with rows matching teams/brks and columns matching rounds.
    Entries are game-identifiers within the same round, with h or a suffix."""
    indices = [ (t,br[0],br[1]) for (t,br) in zip(tms,brks) ]
    entries = dict()
    for r in range(N-1):
        entries["R_"+str(r)] = [ -1 for i in range(N) ]
        
    gamenumber = np.zeros([N,N],dtype=int)
    for i in range(N):
        for j in range(N):
            gamenumber[i][j] = -1
    gamecnt = [ 0 for r in range(N-1)]
    for i in range(N):
        for j in range(i+1,N):
            rndij = rnds[i][j]
            if gamenumber[i][j] == -1:
                gamenumber[i][j] = gamecnt[rndij]
                gamenumber[j][i] = gamecnt[rndij]
                gamecnt[rndij] += 1    
        
    for i in range(N):
        (ti,r,p) = indices[i]
        for j in range(i+1,N):
            (tj,r,p) = indices[j]
            rndij = rnds[i][j]
            Vij,Vji = "h","a"
            if ( (ti+tj)%2==0 and ti < tj ) or ( (ti+tj)%2==1 and ti > tj ):
                Vij,Vji = "a","h"
            gameij = gamenumber[i][j]
            entries["R_"+str(rndij)][i] = str(gameij)+Vij
            entries["R_"+str(rndij)][j] = str(gameij)+Vji
            
    df = pd.DataFrame(entries,index=indices)
    
    #print(df)
            
    return df

In [6]:
def matchfix6mod82110(N:int) -> List:
    """Generate a general set of matches r,i,lst
    stating that in round r team i meets with someone from lst.
    Customized for N = 6 modulo 8.
    Based on trial and error we can fix almost half of all matches."""
    
    if N<=30:
        return []
    
    #print("calling matchfix6mod8-2110")
    
    nr31pairs = (N-10)//4
    nrlow = nr31pairs//2
    nrhigh = nr31pairs - nrlow
    last2brkid  = N//2 + 1 - 2*nrlow
    last2brkrnd = 2*last2brkid
    
    match5 = []
    
    GT1 = True
    GT2 = True
    GB1 = False
    GB2 = False
    BT1 = True
    BT2 = True
    BB1 = False
    BB2 = False
    
    #basic green LL triangles
    for k in range(-1,N//2 - last2brkid,1):  
        r = (last2brkrnd + 2*k)%(N-1)
        #upper
        XL=2
        newi = 10+2*k
        newmi = N//2 - 1
        while newi <= newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                
                if GT1 and r>last2brkrnd and (r-last2brkrnd)%4==2 and r < N-2 and newi <= newmi:
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                
                if GT2 and r>last2brkrnd and (r-last2brkrnd)%4==2 and r < N-2 and newi-2 <= newmi-4:
                    mat = [(r-1)%(N-1),(newi-2+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    
                
            newi += XL
            newmi -= XL
        #lower
        XL=2
        newi = 10+2*k+N//2
        newmi = N - 1
        while newi <= newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
            newi += XL
            newmi -= XL
        
        XL= ((N+1)-(10+2*k+N//2))//2 - 1  #(almost) half of available length map top half to second half
        newi = 10+2*k+N//2
        newmi = N + 1 - XL
        while newi <= newmi and XL > 0:
            for p in range(XL):
                if GB1 and r>last2brkrnd and (r-last2brkrnd)%4==2 and r < N-2  and newi <= newmi-XL:
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                
                if GB2 and r>last2brkrnd and (r-last2brkrnd)%4==2 and r < N-2 and newi+2 <= newmi-4:
                    mat = [(r+1)%(N-1),(newi+2+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                
            newi += XL
            newmi -= XL
    
    # This procedure creates an overall symmetric pattern around the lower half
    for k in range(-1,N//8,1):
        r = (last2brkrnd + 2*k)%(N-1)
        #starting from N//2 + 6, ending in row 4 + 2k
        XL=2
        newi = N//2 + 6
        newmi = N + 4 + 2*k - 1
        while newi < 10+2*k+N//2:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                
            newi += XL
            newmi -= XL
            
    #extend green block by single row below and diagonal above
    #from last2brkrnd - 2 to last2brkrnd + N//4 - 3 (including)
    for r in range(last2brkrnd - 2, last2brkrnd + N//4 - 4,1):
        newmi = N//2 + 1
        newi = 8 + (r-last2brkrnd)
        mat = [r%(N-1),newi,newmi]
        match5.append(mat)
    for r in range(last2brkrnd , last2brkrnd + N//4 - 3,1):
        newmi = N//2 + 1 + 4
        newi = 8 + (r-last2brkrnd) - 2
        mat = [r%(N-1),newi,newmi]
        match5.append(mat)
        
            
    #now extend the middle right triangle by purple blocks
    #for k in range(N//8 + 1,N//2 - last2brkid ,1):
    for k in range(N//8 ,N//2 - last2brkid ,1):
        r = (last2brkrnd + 2*k)%(N-1)
        #starting from last2brkid + 2, ending in row N//2 + 2 +2(k-N//8)
        XL=2
        newi = last2brkid + 2 
        newmi = N//2 + 2 + 2*(k - N//8) - 1 
        while newmi > N//2:  
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                #print(str(mat))
                
                #have same pairing to the right in column r+1; take care for r=N-2
                #TEST PURPLE +1': ok for r-l2brnd = 2 mod 4
                if r < N-2 and (r-last2brkrnd)%4==2 and r >= last2brkrnd+2*(N//8):
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    #print(str(mat))
                    
                if False and r < N-2 and (r-last2brkrnd)%4==2 and newmi > N//2 + 2:
                    mat = [(r-1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    #print(str(mat))
                
            newi += XL
            newmi -= XL
        
    Pairing = True  
    #extend purple part by full row above and diagnal below also over other columns
    firstr = last2brkrnd + N//4 - 3
    for r in range(firstr+2 ,N-1 ,1):
        if r <= firstr+1 or r > firstr+3:
            newi = last2brkid + 1
            newmi = N//2 + 1 + (r - (last2brkrnd + N//4 - 3))
            mat = [r%(N-1),newi,newmi]
            match5.append(mat)
            if not Pairing:
                newi = last2brkid - 1
                newmi = N//2 + 1 + (r - (last2brkrnd + N//4 - 3)) + 4
                mat = [r%(N-1),newi,newmi]
                match5.append(mat)

        else:
            newi = last2brkid + 1
            if r==firstr+2:
                newmi = N//2 + 1 + (r - (last2brkrnd + N//4 - 3)) + 1 
            else:
                newmi = N//2 + 1 + (r - (last2brkrnd + N//4 - 3)) - 1 
            mat = [r%(N-1),newi,newmi]
            match5.append(mat)
            
      
    
    
    #extend the purple triangles by blue blocks at distance 1, skipping first col
    for k in range(N//8 +1 ,N//2 - last2brkid ,1): 
    
        r = (last2brkrnd + 2*k)%(N-1)
        #starting from last2brkid + 2, ending in row N//2 + 2 +2(k-N//8)
        XL=4
        newi = last2brkid - 3
        newmi = N//2 + 4 + 2*(k - N//8)
        for p in range(XL):
            mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
            match5.append(mat)
            
            #extend same 'pairing' to the right/left as with purple:
            if r < N-2 and (r-last2brkrnd)%4==2 and Pairing:
                    mat = [(r-1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    #print(str(mat))
            
       
    #extend the blue blocks by orange triangles at distance 5 from purple
    for k in range(N//8,N//2 - last2brkid ,1):  
        r = (last2brkrnd + 2*k)%(N-1)
        #starting from last2brkid + 2, ending in row N//2 + 2 +2(k-N//8)
        XL=2
        newi = last2brkid + 2 - 7
        newmi = N//2 + 2 + 2*(k - N//8) - 1 + 7
        while newmi < N//2 + last2brkid - 6: 
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                #extend on r+- 1 as in purple
                if r < N-2 and (r-last2brkrnd)%4==2 and Pairing:
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                
            newi -= XL
            newmi += XL
     
    # try to fit in one more triangle cyan
    for k in range(N//8 +1 ,N//2 - last2brkid + 4 ,1):   #going over the boundary, maar toch maar niet
        r = (last2brkrnd + 2*k)
        XL = 2
        sumi = N//2 + last2brkid + 2*(k - N//8) + 3
        newi = 7
        newmi = sumi - newi
        cnt = k-N//8 - 1
        while cnt > 0:
            for p in range(XL):
                mat = [r%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                #print(str(mat))
                #extend on r+- 1 as in purple
                if (r-last2brkrnd)%4==2 and Pairing:  #r < N-2 and 
                    
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                
            newi += XL
            newmi -= XL
            cnt -= 1  
            
    #what remains to be filled are the counterparts for 1,3,6 in rnds > start of purple + 1
    for r in range(last2brkrnd+N//4 - 1,N-1,2):
        mat = [r,1,r]
        match5.append(mat)
        #print(str(mat))
        if r > last2brkrnd+N//4 - 1:
            mat = [r,3,r-2]
            match5.append(mat)
        if r > last2brkrnd+N//4 - 1 + 2:
            mat = [r,6,r-4]
            match5.append(mat)
            
            
    #basic blue part
    for k in range(last2brkid - 6):
        r = last2brkrnd -1 - 2*k
        #upper
        XL=2
        newi = 6
        newmi = N//2 + 1 - 2*k
        if k==0:
            newi += 2
            newmi -= 2
        while newi <= newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                
                if BT1 and r<last2brkrnd-3 and (last2brkrnd-r)%4==1 and r > 11 and newi+2 <= newmi:
                    mat = [(r-1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    #print(str(mat))
                    
                if BT2 and r<last2brkrnd-3 and (last2brkrnd-r)%4==1 and r > 11 and newi+2 <= newmi:
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+2+q)%N for q in range(XL)]
                    match5.append(mat)
                    #print(str(mat))
                    
            newi += XL
            newmi -= XL
        #lower
        XL=2
        newi = 6+N//2
        newmi = N + 1 - 2*k
        while newi <= newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                
                if BB1 and r<last2brkrnd-3 and (last2brkrnd-r)%4==1 and r > 11 and newi+2 <= newmi:
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    
                if BB2 and r<last2brkrnd-3 and (last2brkrnd-r)%4==1 and r > 11 and newi+4 <= newmi:
                    mat = [(r-1)%(N-1),(newi+p)%N]+[(newmi-2+q)%N for q in range(XL)]
                    match5.append(mat)
                    
            newi += XL
            newmi -= XL         
    
    #try pinkify lower blue
    for k in range(4,N//8+1):
        r = last2brkrnd -1 - 2*k
        newi = N//2 + 2   
        newmi = N - 2*k +3   
        XL = 4
        for p in range(XL):
            mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
            match5.append(mat)
            
    #force 0,1 meets above pink and two cols to the left
    for k in range(4,6):  #skip double definitions N//8+1 + 2):
        r = last2brkrnd -1 - 2*k
        if k==4:
            newi = N//2+1
            newmi =  newi - 1
        else:
            newi = N//2 - 2*(k-5)   
            newmi = newi - 2
            
        mat = [r,0,newi]
        match5.append(mat)
        mat = [r,1,newmi]
        match5.append(mat)
        
        #next work ladder down, for all cases:
        pos1 = newmi
        newi = pos1 + 1  #one row below 1
        newmi = N-3
        while newi < N//2 and newmi > last2brkid + N//2:   #for k=4 skips action
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            #print(str(mat))
            newi += 2
            newmi -= 2
        
    print("paths above and below pink" )
    startk = N//8 - 4
    
    for k in range(4,N//8 + 1 ,1): 

        r = (last2brkrnd -3 -2 - 2*k)%(N-1)  # column shift by 2 cmprd to 2 mod 8
        #first ladder up: purple
        maxat = N//2 + 1
        maxto = N - 1 - 2*k + 5
        mat = [r,maxat,maxto%N]
        match5.append(mat)
        #print(str(mat))
            
        newi = maxat - 1
        newmi = maxto + 2
        while newmi <= N:
            mat = [r,newi%N,newmi%N]
            if newmi==N-2 and (N//8-1-k)%4 == 0:
                pass
            else:
                match5.append(mat)
                #print(str(mat))
            newi -= 2
            newmi += 2
        #finishing with newi at row linked to  1
        mat = [r,1,newi%N]
        match5.append(mat)
        #print(str(mat))
            
        #next work ladder down, for all cases:
        newi += 1
        newmi = N-3
        while newi < N//2 and newmi > last2brkid + N//2:   #newmi >= N - 1 - 2*k + 4:
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            #print(str(mat))
            newi += 2
            newmi -= 2       
    
    #extend bottom by interweaving paths left of pink 
    # first the purple part
    lastk = (last2brkrnd - 18)//2 +1  #including column 11
    for k in range(N//8+1 ,lastk+1,1):
        r = (last2brkrnd -3 -2 - 2*k)%(N-1)
        #only ladder up; down will be taken care of later
        maxat = N//2 + 1 - 2*(k-N//8)
        maxto = N//2 + last2brkid + 1 
        mat = [r,maxat+1,maxto]
        match5.append(mat)  #replacing position of the max by 1
        #print(str(mat))
        newi = maxat - 1
        newmi = maxto + 2
        while newi >= last2brkid + 1:   
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            newi -= 2
            newmi += 2
            #print(str(mat))
       
    #next add orange/brown part of interweave; left of pink
    for k in range(-1,N//8-3,1): 
        r = 13 + 2*k
        newi = last2brkid + 2 - 2 +2
        newmi = last2brkid + N//2 +2 + 2*k -2
        while newmi >= last2brkid + N//2:  # - 2 :
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            newi += 2
            newmi -= 2
            #print(str(mat))
        
        mat = [r,newi,newmi+1]
        match5.append(mat)
    
    print("building middle left")
    #supporting green part middle left
    for k in range(0,N//8 - 2):
        r = 11 + 2*k
        XL = 2
        newi = last2brkid + 4 + 2*k
        newmi = N//2 + last2brkid - 3  
        while newi <= newmi and newi <= N//2 + 5:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                #print(str(mat))
                                 
            newi += XL
            newmi -= XL

            
    #add copy shifted N//2
    for k in range(N//8 - 2):
        # test for 78
        r = 11 + 2*k
        XL = 2
        newi = last2brkid + 4 + 2*k + N//2
        newmi = N//2 + last2brkid - 3  + N//2 
        while newi <= newmi and newi <= N//2 + 5 + N//2:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                #print(str(mat))
                
            newi += XL
            newmi -= XL
            
    #finally fill columns 1,3,5,7,9
    
    #beginner's blocks
    
    list1 = [[1,0,1],[3,0,3],[3,1,2],[3,4,N-1],
    [5,1,3],[5,2,4],[7,1,6],[7,2,5],[7,3,4],
    [9,1,8],[9,2,7],[9,3,6],[9,4,5]]
    tuplist = [ tuple(ls) for ls in list1 ]
    list2 = [ [p,(q+N//2)%N,(r+N//2)%N] for (p,q,r) in tuplist ]
    match5.extend(list1)
    match5.extend(list2)
    
    #""" 
    #orange cells, interspaced
    mat = [1,N//2-3,N//2+4]
    match5.append(mat)
    mat = [3,N//2-2,N//2+4+2]
    match5.append(mat)
    for r in [1,3]:
        newi = N//2- 4
        newmi = N//2 + r + 5
        while newi >= last2brkid + 3:
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            newi -= 2
            newmi += 2
            
    for r in [5,7,9]:
        newi = N//2
        newmi = N//2 + 1 + r
        while newi >= last2brkid + 3:
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            if r==9:
                mat = [r,(newi-3)%N,(newmi-1)%N]
                if newi == last2brkid + 3:
                    mat = [r,(newi-2)%N,(newmi-1)%N]
            if r==7:
                mat = [r,(newi-3)%N,(newmi-3)%N,(newmi+3)%N]
                if newi == last2brkid + 3:
                    mat = [r,(newi-2)%N,(newmi-3)%N,(newmi+3)%N]
            if r==5:
                mat = [r,(newi-3)%N,(newmi+1)%N]
                if newi == last2brkid + 3:
                    mat = [r,(newi-2)%N,(newmi+1)%N]
            if r>= 5:
                match5.append(mat)
            newi -= 2
            newmi += 2
    #""" 
            
    #special for r = 5:
    r = 5
    newi = last2brkid
    newmi = last2brkid + N//2 - 5
    while newi < newmi:
        mat = [r,newi,newmi,newmi+1]
        match5.append(mat)
        mat = [r,newi+1,newmi,newmi+1]
        match5.append(mat)
        mat = [r+1,newi,newmi,newmi+1]
        match5.append(mat)
        mat = [r+1,newi+1,newmi,newmi+1]
        match5.append(mat)
        newi += 2
        newmi -= 2
            
        
    return match5


In [7]:
def matchfixSym2110(N:int) -> List:
    """Generate a symmetrical set of matches r,i,lst
    stating that in round r team i meets with someone from lst.
    First focus on complete schedule fo Q3Q1 and one for Q2Q4"""
    
    #code below works only for N=2 mod 8
    if N%8 == 6:
        return matchfix6mod82110(N)
    else:
        pass
    
    if N<=30:
        return []
    
    nr31pairs = (N-10)//4
    nrlow = nr31pairs//2
    nrhigh = nr31pairs - nrlow
    last2brkid  = N//2 + 1 - 2*nrlow
    last2brkrnd = 2*last2brkid
    
    match5 = []
    
    GT1 = True
    GT2 = True
    GB1 = False
    GB2 = False
    BT1 = True
    BT2 = True
    BB1 = False
    BB2 = False
    
    #basic green LL triangles
    for k in range(-1,N//2 - last2brkid,1):  
        r = (last2brkrnd + 2*k)%(N-1)
        #upper
        XL=2
        newi = 8+2*k
        newmi = N//2 - 1
        while newi <= newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                
                if GT1 and r>last2brkrnd and (r-last2brkrnd)%4==2 and r < N-2 and newi <= newmi:
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    if newi == 8+2*k:
                        mat = [(r+1)%(N-1),(newi-2+p)%N]+[(newmi+2+q)%N for q in range(XL)]
                        match5.append(mat)
                
                if GT2 and r>last2brkrnd and (r-last2brkrnd)%4==2 and r < N-2 and newi-2 <= newmi-4:
                    mat = [(r-1)%(N-1),(newi-2+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    if newi == 8+2*k:
                        mat = [(r-1)%(N-1),(newi-2+p-2)%N]+[(newmi+q+2)%N for q in range(XL)]
                        match5.append(mat)
                    
                
            newi += XL
            newmi -= XL
        #lower
        XL=2
        newi = 8+2*k+N//2
        newmi = N//2 - 1 + N//2
        while newi <= newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat) 
                
                
                if GB1 and r>last2brkrnd and (r-last2brkrnd)%4==2 and r < N-2  and newi <= newmi-4:
                    mat = [(r-1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                
                if GB2 and r>last2brkrnd and (r-last2brkrnd)%4==2 and r < N-2 and newi+2 <= newmi-4:
                    mat = [(r+1)%(N-1),(newi+2+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                
            newi += XL
            newmi -= XL
            
    #extend upper green by two pairs
    for k in range(0,N//8,1): 
        #break
        r = (last2brkrnd + 2*k)%(N-1)
        XL = 2
        newi = 8+2*k - 2
        newmi = N//2 - 1 + 2
        for p in range(XL):
            mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
            match5.append(mat)
            # add r+1 TEST
            if (r-last2brkrnd)%4==2:
                mat = [r+1,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
            # add r-1 TEST98
            if (r-last2brkrnd)%4==2:
                mat = [r-1,(newi-2+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                
    #extend by two pink blocks of 4
    for k in range(2,N//8,1): 
        #break
        r = (last2brkrnd + 2*k)%(N-1)
        XL = 4
        newi = 8+2*k - 2 - 4
        newmi = N//2 - 1 + 2 + 2
        for p in range(XL):
            mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
            match5.append(mat)
            
    #extend by remaining green pairs
    for k in range(3,N//8,1): 
        #break
        r = (last2brkrnd + 2*k)%(N-1)
        XL = 2
        newi = 8+2*k - 2 - 4 - 2
        newmi = N//2 - 1 + 2 + 2 + 4
        while newi >= 6:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                #TEST SPEEDUP for 2 mod 8 right corner
                #extend on r+- 1 as in purple or 6 mod 8
                if (r + N-1 -last2brkrnd)%4==2 and k <N//8-1:
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
            newi -= 2
            newmi += 2
            
    #fix 3,5,2,1,4 sequence form lst2brnd + 4 tot n//4-2
    for r in range(last2brkrnd + 4, last2brkrnd + N//4 - 2,2):
        fst = N//2 + 3 + (r-last2brkrnd)
        mats=[[r,3,fst],[r,5,fst+1],[r,2,fst+2],[r,1,fst+3],[r,4,fst+4]]
        match5.extend(mats)
        
    #fix 6,7,8,.. to N//2+last2brkid -3 vanaf last2brkrnd + N//4 -2 + 4
    #TEST
    # N=74 fstr = 44 + 18 -2 +2 = 62
    
    fstr = last2brkrnd + N//4 -2 + 2
    for r in range(fstr,N-1,1):
        mat = [r,6+r-fstr,N//2+last2brkid -3]
        match5.append(mat)
        if (r-fstr)%2 == 0 and False:
            mat = [r,6+r-fstr - 1,N//2+last2brkid -3 + 4]
            match5.append(mat)
     
    

    #basic blue UR triangles
    for k in range(-1,N//2 - last2brkid,1):  
        r = (last2brkrnd -3 - 2*k)%(N-1)
        #upper
        XL=2
        newi = 6
        newmi = N//2 - 3 - 2*k
        while newi <= newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                
                if BT1 and r<last2brkrnd-3 and (last2brkrnd-r)%4==1 and r > 11 and newi+2 <= newmi:
                    mat = [(r-1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    #print(str(mat))
                    
                if BT2 and r<last2brkrnd-3 and (last2brkrnd-r)%4==1 and r > 11 and newi+2 <= newmi:
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+2+q)%N for q in range(XL)]
                    match5.append(mat)
                    #print(str(mat))
                
            newi += XL
            newmi -= XL
        #lower
        XL=2
        newi = 6+N//2
        newmi = N - 3 - 2*k
        while newi <= newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                
                if BB1 and r<last2brkrnd-3 and (last2brkrnd-r)%4==1 and r > 11 and newi+2 <= newmi:
                    mat = [(r+1)%(N-1),(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                    match5.append(mat)
                    
                if BB2 and r<last2brkrnd-3 and (last2brkrnd-r)%4==1 and r > 11 and newi+4 <= newmi:
                    mat = [(r-1)%(N-1),(newi+p)%N]+[(newmi-2+q)%N for q in range(XL)]
                    match5.append(mat)
                
            
            newi += XL
            newmi -= XL
    
    #return match5 

    #extend bottom blue by two pinks -- skipping k=-1,0,1
    for k in range(2,N//8,1):   
        r = (last2brkrnd -3 - 2*k)%(N-1)
        XL = 4
        newi = 2 + N//2
        newmi = N - 1 - 2*k
        for p in range(XL):
            mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
            match5.append(mat)
            
    #extend bottom by interweaving paths above pink
    for k in range(3,N//8 + 1,1):     #TEST +1
        r = (last2brkrnd -3 - 2*k)%(N-1)
        #first ladder up: purple
        maxat = N//2 + 1
        maxto = N - 1 - 2*k + 5
        mat = [r,maxat,maxto]
        match5.append(mat)
        newi = maxat - 1
        newmi = maxto + 2
        while newmi <= N:
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            newi -= 2
            newmi += 2
        #finishing with newi at row linked to  1
        mat = [r,1,newi%N]
        match5.append(mat)
        #next work ladder down:
        newi += 1
        newmi = N-3
        while newmi >= N - 1 - 2*k + 4:
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            newi += 2
            newmi -= 2
            
    #extend bottom by interweaving paths left of pink
    # first the purple part
    lastk = (last2brkrnd - 16)//2 +1  #including column 11
    for k in range(N//8+1,lastk+1,1):
        #print(str(k))
        r = (last2brkrnd -3 - 2*k)%(N-1)
        #print(str(r))
        #only ladder up
        maxat = N//2 + 1 - 2*(k-N//8)
        maxto = N//2 + last2brkid + 1 
        mat = [r,maxat+1,maxto]
        match5.append(mat)  #replacing position of the max by 1
        #print(str(mat))
        newi = maxat - 1
        newmi = maxto + 2
        while newi >= last2brkid + 1:   #TEST changed 2 to 1
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            newi -= 2
            newmi += 2
            #print(str(mat))
            
    #next add orange/brown part of interweave
    for k in range(-1,N//8-3,1):  
        r = 13 + 2*k
        newi = last2brkid + 2
        newmi = last2brkid + N//2 +2 + 2*k
        while newmi >= last2brkid + N//2 :
            mat = [r,newi%N,newmi%N]
            match5.append(mat)
            newi += 2
            newmi -= 2
            #print(str(mat))
            
    
    #fill left part from column 5 onward upto r = (last2brkrnd -3 - 2*k)%(N-1) for k=N//8
    # kcnt = (N//2 + 6 - (last2brkid + 1))//2
    #for k in range(0,N//8+2,1):  #N//4  ipv 4
    kcnt = (N//2 + 6 - (last2brkid + 1))//2
    for k in range(0,kcnt,1):
        r = 5+2*k
            
        XL = 2
        #middle section first
        newi = last2brkid + 2 +2*k
        newmi = last2brkid + N//2 - 3 - XL
        
        #TEST FOR MORE that already showed
        if k>0 and k < kcnt-1:
            newi -= 2
            newmi += 2
        
  
        while newi < newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
            newi += 2
            newmi -= 2
            
        #the same N//2 shifted up/downward to get the range part top and botttom
        newi = last2brkid + 2 +2*k +N//2
        newmi = last2brkid + N//2 - 3 - XL +N//2
        while newi < newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
            newi += 2
            newmi -= 2
    
    #remaining orange part in top and bottom quarter , columns 1,3 :
    for k in range(0,2,1):
        r = (1+2*k)%(N-1)
        XL = 2
        newi = 2*k + 1
        newmi = N - 1
        if k==0:
            newi += 2
            newmi -= 2
            
        while last2brkid + N//2  < newmi:
            for p in range(XL):
                mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                match5.append(mat)
                #print(str(mat))
            newi += 2
            newmi -= 2
            
            
    #Proceed to RHS starting from last2brk + N//8
    for k in range(-1,N//2 - last2brkid +5 ,1):   #+2,+3 move to the START
        r = (last2brkrnd + 2*k)%(N-1)
        
        lasti = 2*last2brkid+1-N//2 + 2*k    #it is actually the top row index
        
        
        #purple triangles top and bottom, not going beyond column N-2
        kstop = N//8 - 1 
        if k > kstop and k <= (N-2 - last2brkrnd)//2:
                newi = 1
                newmi = lasti + N//2 - 2
                XL = 2
                while newmi > last2brkid + N//2 :  
                    for p in range(XL):
                        mat = [r,(newi+p)%N]+[(newmi+q)%N for q in range(XL)]
                        match5.append(mat)
                    newi += 2
                    newmi -= 2
        
    return match5
    

In [ ]:
for N in range(18,111,4):
    match5 = matchfixSym2110(N)
    aa,bb,c38 = fpbreak(N,match=match5,part=False,allfree=False)
    df58 = build_df(N,aa,bb,c38)
    df58.to_excel(f"results/sched{N}.xlsx")